In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/Othercomputers/陈成的thinkpad/software-cup")
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.4 MB/s 
     |████████████████████████████████| 12.6 MB 19.0 MB/s 
     |████████████████████████████████| 15.3 MB 415 kB/s 
     |████████████████████████████████| 711.8 MB 5.4 kB/s 
     |████████████████████████████████| 9.5 MB 46.4 MB/s 
     |████████████████████████████████| 5.3 MB 49.4 MB/s 
     |████████████████████████████████| 215 kB 56.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting u

In [3]:
os.chdir("/content/drive/Othercomputers/陈成的thinkpad/software-cup/task1_2_3")

In [4]:
import gc

import numpy as np
import pandas as pd
import sklearn
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
from xgboost import XGBClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ._gradient_boosting import predict_stages


In [5]:
def saved(df, file):
    df.to_csv(file, header=False, index=False)
    print('saved')

# 任务一，求每个用户的年平均缴费次数和平均金额

In [6]:
power = pd.read_excel("../dataset/cph.xlsx")
power["缴费日期"] = pd.to_datetime(power["缴费日期"])

def getYear(df, date):
    year = []
    for i in df[date]:
        year.append(i.year)
    return year


power["year"] = getYear(power, "缴费日期")
power
power_user = power.groupby("用户编号")
user_list = list(power_user)
user_id = []
user_chargeDate_mean = []
user_chargeMoney_mean = []
user_charge_Money_Date = []
for i in user_list:
    user = pd.DataFrame(i[1])
    id = user.用户编号.unique()[0]
    len = user["year"].nunique()
    charge_Date_mean = user.缴费日期.nunique() / len
    charge_Moeny_mean = user["缴费金额（元）"].sum() / len
    charge_Money_Date = user["缴费金额（元）"].sum() / charge_Date_mean
    user_id.append(id)
    user_chargeDate_mean.append(charge_Date_mean)
    user_chargeMoney_mean.append(charge_Moeny_mean)
    user_charge_Money_Date.append(charge_Money_Date)

power_mean = pd.DataFrame()
power_mean["用户编码"] = user_id
power_mean["年平均缴费次数"] = user_chargeDate_mean
power_mean["年平均缴费金额"] = user_chargeMoney_mean
power_mean["平均每次缴费金额"] = user_charge_Money_Date
#平均每次缴费金额
# saved(power_mean,'Task1.csv')
power_mean

,用户编码,年平均缴费次数,年平均缴费金额,平均每次缴费金额
0,1000000001,4.0,493.5,246.750000
1,1000000002,3.5,245.0,140.000000
2,1000000003,3.5,590.0,337.142857
3,1000000004,4.0,310.5,155.250000
4,1000000005,3.5,750.0,428.571429
...,...,...,...,...
95,1000000096,3.5,458.0,261.714286
96,1000000097,3.0,324.5,216.333333
97,1000000098,3.0,367.5,245.000000
98,1000000099,3.0,426.5,284.333333


In [7]:
charge_Date_mean

3.5

# 任务二 客户分类

In [8]:
count = power_mean.年平均缴费次数.mean()
money = power_mean.年平均缴费金额.mean()

high_value = pd.DataFrame(columns=['用户编码', '年平均缴费次数', '年平均缴费金额', '平均每次缴费金额'])
low_value = pd.DataFrame(columns=['用户编码', '年平均缴费次数', '年平均缴费金额', '平均每次缴费金额'])
normal = pd.DataFrame(columns=['用户编码', '年平均缴费次数', '年平均缴费金额', '平均每次缴费金额'])
potency = pd.DataFrame(columns=['用户编码', '年平均缴费次数', '年平均缴费金额', '平均每次缴费金额'])

client = pd.DataFrame(columns=['用户编码', '年平均缴费次数', '年平均缴费金额', '平均每次缴费金额', '是否为高价值'])

for row in power_mean.iterrows():
    count_mean = row[1]["年平均缴费次数"]
    money_mean = row[1]["年平均缴费金额"]
    money_count = row[1]["平均每次缴费金额"]
    id = str(int(row[1]["用户编码"]))

    if (count_mean > count and money_mean > money):
        high_value = high_value.append(
            {'用户编码': id, '年平均缴费次数': count_mean, '年平均缴费金额': money_mean, '平均每次缴费金额': money_count}, ignore_index=True)

        client = client.append(
            {'用户编码': id, '年平均缴费次数': count_mean, '年平均缴费金额': money_mean, '平均每次缴费金额': money_count, '是否为高价值': 1},
            ignore_index=True)
    elif (count_mean <= count and money_mean <= money):
        low_value = low_value.append(
            {'用户编码': id, '年平均缴费次数': count_mean, '年平均缴费金额': money_mean, '平均每次缴费金额': money_count}, ignore_index=True)

        client = client.append(
            {'用户编码': id, '年平均缴费次数': count_mean, '年平均缴费金额': money_mean, '平均每次缴费金额': money_count, '是否为高价值': 0},
            ignore_index=True)
    elif (count_mean > count and money_mean <= money):
        normal = normal.append({'用户编码': id, '年平均缴费次数': count_mean, '年平均缴费金额': money_mean, '平均每次缴费金额': money_count},
                               ignore_index=True)

        client = client.append(
            {'用户编码': id, '年平均缴费次数': count_mean, '年平均缴费金额': money_mean, '平均每次缴费金额': money_count, '是否为高价值': 0},
            ignore_index=True)
    elif (count_mean <= count and money_mean > money):
        potency = potency.append({'用户编码': id, '年平均缴费次数': count_mean, '年平均缴费金额': money_mean, '平均每次缴费金额': money_count},
                                 ignore_index=True)

        client = client.append(
            {'用户编码': id, '年平均缴费次数': count_mean, '年平均缴费金额': money_mean, '平均每次缴费金额': money_count, '是否为高价值': 0},
            ignore_index=True)

print("高价值型客户：\n{}；\n低价值型客户：\n{}；\n大众型客户：\n{}；\n潜力型客户：\n{}".format(high_value, low_value, normal, potency))
saved(high_value, 'Task2_high.csv')
saved(low_value, 'Task2_low.csv')
saved(normal, 'Task2_normal.csv')
saved(potency, 'Task2_potency.csv')
client



#38 23 33 2

高价值型客户：
          用户编码  年平均缴费次数  年平均缴费金额    平均每次缴费金额
0   1000000001      4.0    493.5  246.750000
1   1000000003      3.5    590.0  337.142857
2   1000000005      3.5    750.0  428.571429
3   1000000006      3.5    505.0  288.571429
4   1000000007      3.5    553.0  316.000000
5   1000000008      3.5    537.0  306.857143
6   1000000009      3.5    485.0  277.142857
7   1000000010      4.0    386.5  193.250000
8   1000000011      3.5    374.0  213.714286
9   1000000012      3.5    455.5  260.285714
10  1000000013      3.5    417.5  238.571429
11  1000000014      3.5    374.5  214.000000
12  1000000016      4.0    392.0  196.000000
13  1000000017      3.5    390.0  222.857143
14  1000000020      3.5    449.5  256.857143
15  1000000021      3.5    378.5  216.285714
16  1000000024      3.5    452.0  258.285714
17  1000000026      3.5    388.5  222.000000
18  1000000034      3.5    360.5  206.000000
19  1000000035      3.5    384.5  219.714286
20  1000000039      3.5    375.0  214.285714
21

,用户编码,年平均缴费次数,年平均缴费金额,平均每次缴费金额,是否为高价值
0,1000000001,4.0,493.5,246.750000,1
1,1000000002,3.5,245.0,140.000000,0
2,1000000003,3.5,590.0,337.142857,1
3,1000000004,4.0,310.5,155.250000,0
4,1000000005,3.5,750.0,428.571429,1
...,...,...,...,...,...
95,1000000096,3.5,458.0,261.714286,1
96,1000000097,3.0,324.5,216.333333,0
97,1000000098,3.0,367.5,245.000000,0
98,1000000099,3.0,426.5,284.333333,0


In [9]:
high_value, low_value, normal, potency

(          用户编码  年平均缴费次数  年平均缴费金额    平均每次缴费金额
 0   1000000001      4.0    493.5  246.750000
 1   1000000003      3.5    590.0  337.142857
 2   1000000005      3.5    750.0  428.571429
 3   1000000006      3.5    505.0  288.571429
 4   1000000007      3.5    553.0  316.000000
 5   1000000008      3.5    537.0  306.857143
 6   1000000009      3.5    485.0  277.142857
 7   1000000010      4.0    386.5  193.250000
 8   1000000011      3.5    374.0  213.714286
 9   1000000012      3.5    455.5  260.285714
 10  1000000013      3.5    417.5  238.571429
 11  1000000014      3.5    374.5  214.000000
 12  1000000016      4.0    392.0  196.000000
 13  1000000017      3.5    390.0  222.857143
 14  1000000020      3.5    449.5  256.857143
 15  1000000021      3.5    378.5  216.285714
 16  1000000024      3.5    452.0  258.285714
 17  1000000026      3.5    388.5  222.000000
 18  1000000034      3.5    360.5  206.000000
 19  1000000035      3.5    384.5  219.714286
 20  1000000039      3.5    375.0 

# 任务三，预测高价值top5

In [10]:
from sklearn import model_selection

# 将数据集拆分为训练集和测试集
X = client[['年平均缴费次数', '年平均缴费金额', '平均每次缴费金额']]
y = client[['用户编码', '是否为高价值']]
X, X_test, y, y_test = model_selection.train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.1)
Y_test = y_test
y = y['是否为高价值']
y_train = y_train['是否为高价值']
y_valid = y_valid['是否为高价值']
y_test = y_test['是否为高价值']

In [11]:
y_valid = y_valid.to_numpy().astype(np.float64)
y_test = y_test.to_numpy().astype(np.float64)
print(y_test)

[1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0.]


In [12]:
# Initializing Classifiers
clf1 = XGBClassifier()
clf2 = GradientBoostingClassifier()
clf3 = CatBoostClassifier()
clf4 = LGBMClassifier()
eclf = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3, clf4], weights=[2, 1, 1, 1], voting='soft')

eclf.fit(X_train, y_train)

gc.collect()

oof_prob = eclf.predict_proba(X_valid)[:, 1]


def find_best_threshold(y_valid, oof_prob):
    best_auc = 0

    for th in tqdm([i / 100 for i in range(30, 70)]):
        oof_prob_copy = oof_prob.copy()
        oof_prob_copy[oof_prob_copy >= th] = 1
        oof_prob_copy[oof_prob_copy < th] = 0

        auc = sklearn.metrics.roc_auc_score(y_valid, oof_prob)

        if auc > best_auc:
            best_th = th
            best_auc = auc

        gc.collect()

    return best_th, best_auc


best_th, best_auc = find_best_threshold(y_valid, oof_prob)
print(best_th, best_auc)

eclf1 = EnsembleVoteClassifier(clfs=[clf1, clf2, clf3], weights=[2, 1, 1], voting='soft')
eclf1.fit(X, y)
y_pre = eclf.predict_proba(X_test)[:, 1]
# test=pd.read_csv(data_path_3+'x_test_B.csv')
# res = test[['id']]
# res['y'] = y_pre
# res.loc[res['y'] >= best_th, 'y'] = 1
# res.loc[res['y'] < best_th, 'y'] = 0
# res.to_csv('0214_bagging_lgb.csv',index = False)
# print('saved')

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:1302: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.object)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:1489: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_mask = np.ones((n_samples, ), dtype=np.bool)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:1162: Depr

Learning rate set to 0.00335
0:	learn: 0.6879725	total: 47.2ms	remaining: 47.2s
1:	learn: 0.6833458	total: 47.8ms	remaining: 23.9s
2:	learn: 0.6781754	total: 48.9ms	remaining: 16.3s
3:	learn: 0.6743283	total: 49.9ms	remaining: 12.4s
4:	learn: 0.6688156	total: 50.7ms	remaining: 10.1s
5:	learn: 0.6647604	total: 51.6ms	remaining: 8.54s
6:	learn: 0.6593466	total: 52.4ms	remaining: 7.44s
7:	learn: 0.6549589	total: 53.2ms	remaining: 6.6s
8:	learn: 0.6492052	total: 53.9ms	remaining: 5.93s
9:	learn: 0.6446711	total: 54.6ms	remaining: 5.41s
10:	learn: 0.6401237	total: 55.3ms	remaining: 4.97s
11:	learn: 0.6349967	total: 56.1ms	remaining: 4.62s
12:	learn: 0.6301427	total: 57.1ms	remaining: 4.33s
13:	learn: 0.6250087	total: 58.1ms	remaining: 4.09s
14:	learn: 0.6197657	total: 58.8ms	remaining: 3.86s
15:	learn: 0.6153346	total: 59.5ms	remaining: 3.66s
16:	learn: 0.6112474	total: 60.7ms	remaining: 3.51s
17:	learn: 0.6067846	total: 61.1ms	remaining: 3.33s
18:	learn: 0.6019988	total: 61.5ms	remaining: 

100%|██████████| 40/40 [00:05<00:00,  7.73it/s]
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:1302: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.object)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:1489: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_mask = np.ones((n_samples, ), dtype=np.bool)
/usr/local/lib/python3.7/dist-packages/

0.3 1.0


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:1162: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/gradient_boosting.py:1162: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert sample_mask.dtype == np.bool
/usr/local/lib/pytho

Learning rate set to 0.003504
0:	learn: 0.6872509	total: 1.89ms	remaining: 1.89s
1:	learn: 0.6820281	total: 2.35ms	remaining: 1.17s
2:	learn: 0.6765236	total: 3.37ms	remaining: 1.12s
3:	learn: 0.6716571	total: 4.32ms	remaining: 1.07s
4:	learn: 0.6662642	total: 5.18ms	remaining: 1.03s
5:	learn: 0.6624546	total: 6ms	remaining: 995ms
6:	learn: 0.6573688	total: 6.82ms	remaining: 967ms
7:	learn: 0.6526367	total: 7.59ms	remaining: 942ms
8:	learn: 0.6478181	total: 8.14ms	remaining: 896ms
9:	learn: 0.6428728	total: 8.79ms	remaining: 870ms
10:	learn: 0.6380595	total: 9.41ms	remaining: 846ms
11:	learn: 0.6326962	total: 10.1ms	remaining: 830ms
12:	learn: 0.6274643	total: 10.8ms	remaining: 819ms
13:	learn: 0.6221397	total: 11.5ms	remaining: 808ms
14:	learn: 0.6170258	total: 12.2ms	remaining: 800ms
15:	learn: 0.6126683	total: 12.9ms	remaining: 791ms
16:	learn: 0.6079449	total: 13.6ms	remaining: 785ms
17:	learn: 0.6032049	total: 18ms	remaining: 981ms
18:	learn: 0.5987289	total: 18.5ms	remaining: 954

In [13]:
high = pd.DataFrame()
res = Y_test[['用户编码']]
res['y'] = y_pre
for row in res.iterrows():
    user_id = row[1][0]
    p = row[1][1]
    if p > best_th:
        high = high.append({'用户编码': user_id, '高价值概率': p}, ignore_index=True)

high.sort_values("高价值概率", ascending=False, inplace=True)
high_5 = high[:5].reset_index(drop=True)
high_5
saved(high_5, 'Task3.csv')

saved


In [14]:
Y_test

,用户编码,是否为高价值
5,1000000006,1
34,1000000035,1
16,1000000017,1
4,1000000005,1
60,1000000061,0
69,1000000070,0
49,1000000050,0
98,1000000099,0
33,1000000034,1
93,1000000094,0


In [15]:
high.shape[0]

10

In [16]:
y_pre

array([0.97655277, 0.9876536 , 0.98768685, 0.97579264, 0.00876287,
       0.01206484, 0.00874365, 0.67714949, 0.78416082, 0.00779979,
       0.0081564 , 0.98909549, 0.9876035 , 0.00849193, 0.00672442,
       0.78416082, 0.97660674, 0.00924135, 0.00984169, 0.01140367])

In [17]:
# y_pre[y_pre >= best_th] = 1
# y_pre[y_pre < best_th] = 0
recall = sklearn.metrics.recall_score(y_test, np.round(y_pre))
precision = sklearn.metrics.precision_score(y_test, np.round(y_pre))
f2 = 5 * recall * precision / (4 * precision + recall)
print(precision)

0.9
